In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
            return result

In [5]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}


In [6]:
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [10]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\anush\\Downloads\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [11]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [12]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [13]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [14]:
#Intialize MLP Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [15]:
#fit the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [16]:
#Predict the test data
y_pred=model.predict(x_test)

In [17]:
y_pred

array(['happy', 'calm', 'happy', 'happy', 'disgust', 'calm', 'calm',
       'disgust', 'calm', 'happy', 'happy', 'calm', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'disgust', 'disgust', 'calm',
       'disgust', 'disgust', 'disgust', 'calm', 'happy', 'happy', 'calm',
       'happy', 'calm', 'calm', 'happy', 'disgust', 'happy', 'calm',
       'happy', 'calm', 'calm', 'fearful', 'calm', 'calm', 'happy',
       'calm', 'calm', 'calm', 'happy', 'disgust', 'disgust', 'calm',
       'calm', 'happy', 'fearful', 'fearful', 'fearful', 'happy', 'happy',
       'fearful', 'calm', 'happy', 'calm', 'calm', 'disgust', 'calm',
       'happy', 'calm', 'disgust', 'calm', 'calm', 'calm', 'disgust',
       'happy', 'happy', 'fearful', 'fearful', 'fearful', 'fearful',
       'disgust', 'fearful', 'happy', 'calm', 'fearful', 'disgust',
       'calm', 'fearful', 'calm', 'disgust', 'calm', 'disgust', 'happy',
       'happy', 'disgust', 'disgust', 'happy', 'calm', 'happy', 'disgust',
       'disgu

In [19]:
#Calculate the accuracy
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.88%
